In [3]:
import pandas as pd
import numpy as np

In [4]:
np.random.seed(24)
df = pd.DataFrame({'A': np.linspace(1, 10, 10)})
df = pd.concat([df, pd.DataFrame(np.random.randn(10, 4), columns=list('BCDE'))], axis=1)
df.iloc[0, 2] = np.nan

In [5]:
#df.style.highlight_null().render().split('\n')[:10]

In [6]:
def color_negative_red(val):
    color, fontstyle = ('rebeccapurple','600') if val < 0 else ('black', '400')
    return 'color: %s; font-weight: %s;' % (color, fontstyle)

In [7]:
s = df.head().style.applymap(color_negative_red);s

,A,B,C,D,E
0,1,1.32921,nan,-0.31628,-0.99081
1,2,-1.07082,-1.43871,0.564417,0.295722
2,3,-1.6264,0.219565,0.678805,1.88927
3,4,0.961538,0.104011,-0.481165,0.850229
4,5,1.45342,1.05774,0.165562,0.515018


In [8]:
df.tail().style.\
    applymap(color_negative_red).\
    apply(highlight_max)

NameError: name 'highlight_max' is not defined

In [ ]:
def highlight_max(s):
    is_max = s == s.max()
    #print(is_max)
    return ['border:2px solid #FFA918;' if v else '' for v in is_max]

In [ ]:
df.head().style.apply(highlight_max)

In [ ]:
def highlight_max2(data, color='yellow'):
    attr = 'background-color: {}'.format(color)
    #print(data) #whole df data-> 2d
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None) 
        is_max = data == data.max().max()
        #print(is_max) basically the checking whole data for max data
    return pd.DataFrame(np.where(is_max, attr, ''), index=data.index, columns=data.columns)

In [ ]:
"""
Styler.applymap(func) for elementwise styles
Styler.apply(func, axis=0) for columnwise styles
Styler.apply(func, axis=1) for rowwise styles
Styler.apply(func, axis=None) for tablewise styles
"""
#in this case, table wise to check data
df.tail().style.apply(highlight_max2, color='darkorange', axis=None)

In [ ]:
##narrow down the applying columns
df.head(3).style.apply(highlight_max, subset=['B', 'C', 'D'])

In [ ]:
df.head().style.applymap(color_negative_red, subset=pd.IndexSlice[2:5, ['B', 'D']])

In [ ]:
df.head(3).style.format("{:.2%}")

In [ ]:
##specifying columns
df.head(3).style.format({'B': "{:0<4.0f}", 'D': '{:+.2f}'})

In [ ]:
df.head(3).style.format({"B": lambda x: "±{:.2f}".format(abs(x))})

In [ ]:
df.head(3).style.highlight_null(null_color='red')

In [10]:
##you can create headmap
import seaborn as sns

In [ ]:
cm = sns.light_palette("blue", as_cmap=True)
df.head(5).style.background_gradient(cmap=cm)

In [ ]:
df.loc[:4].style.background_gradient(cmap='viridis')

In [ ]:
def borderwhite(data):
    return 'border:2px solid white;'

In [ ]:
# Compress the color range
(df.loc[:4]
    .style
    .applymap(borderwhite)
    .background_gradient(cmap='viridis', low=.5, high=0)
    .highlight_null('red'))

In [ ]:
#easy way to light max and min
df.tail(5).style.highlight_max(axis=0)

In [ ]:
df.tail(5).style.highlight_min(axis=1, color='lightgreen')

In [ ]:
df.head(3).style.set_properties(**{'background-color': 'black',
                           'color': 'lawngreen',
                           'border': '1px solid white'})

In [ ]:
df.sort_values(['B'])\
    .head()\
    .style.bar(subset=['A', 'B'], color='#d65f5f')

In [ ]:
df.head(5).style.bar(subset=['A', 'B','C'], align='mid', color=['#d65f5f', '#5fba7d'])

In [ ]:
import pandas as pd
from IPython.display import HTML

# Test series
test1 = pd.Series([-100,-60,-30,-20], name='All Negative')
test2 = pd.Series([10,20,50,100], name='All Positive')
test3 = pd.Series([-10,-5,0,90], name='Both Pos and Neg')

head = """
<table>
    <thead>
        <th>Align</th>
        <th>All Negative</th>
        <th>All Positive</th>
        <th>Both Neg and Pos</th>
    </thead>
    </tbody>

"""

aligns = ['left','zero','mid']
for align in aligns:
    row = "<tr><th>{}</th>".format(align)
    for serie in [test1,test2,test3]:
        s = serie.copy()
        #print(s)
        s.name = ''
        row += "<td>{}</td>"\
            .format(s.to_frame()\
                    .style.bar(align=align,
                               color=['#d65f5f', '#5fba7d'],
                               width=100).render()) #testn['width']
    row += '</tr>'
    head += row

head += """
</tbody>
</table>"""
HTML(head)

In [ ]:
style1 = df.head().style.applymap(color_negative_red); style1

In [ ]:
#you can reuse style from other DF
df2 = -df.head()
style2 = df2.style
style2.use(style1.export())
style2

In [ ]:
with pd.option_context('display.precision', 2):
    html = (df.style
              .applymap(color_negative_red)
              .apply(highlight_max))
html
#or just 
df.tail().style\
  .applymap(color_negative_red)\
  .apply(highlight_max)\
  .set_precision(2)
### You can always use df.round(2).style 
### if you’d prefer to round from the start.

In [ ]:
## you can use caption
df.head().style.set_caption('Colormaps, with a caption.')\
    .background_gradient(cmap=cm)

In [30]:
def hover(hover_color="#ffff99"):
    return dict(selector="tr:hover",
                props=[("background-color", "%s" % hover_color)])
styles = [
    hover(),
    dict(selector="th", props=[("font-size", "150%"), ("text-align", "center")]),
    dict(selector="caption", props=[("caption-side", "bottom")])
]
(df.head().style.set_table_styles(styles).set_caption("Hover to highlight."))

,A,B,C,D,E
0,1,1.32921,nan,-0.31628,-0.99081
1,2,-1.07082,-1.43871,0.564417,0.295722
2,3,-1.6264,0.219565,0.678805,1.88927
3,4,0.961538,0.104011,-0.481165,0.850229
4,5,1.45342,1.05774,0.165562,0.515018


In [ ]:
#hide ugly index
df.head(3).style.hide_index()

In [ ]:
df.head(3).style.hide_columns(['C','D'])

In [16]:
from IPython.html import widgets
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
@widgets.interact
def f(h_neg=(0, 0, 1), h_pos=(0, 512), s=(0., 181.9), l=(0., 121.9)):
    return df.head().style.background_gradient(
        cmap=sns.palettes.diverging_palette(h_neg=h_neg, h_pos=h_pos, s=s, l=l,as_cmap=True)
    )

interactive(children=(IntSlider(value=0, description='h_neg', max=0), IntSlider(value=256, description='h_pos', max=512), FloatSlider(value=90.95, description='s', max=181.9), FloatSlider(value=60.95, description='l', max=121.9), Output()), _dom_classes=('widget-interact',))

In [2]:
def f(x):
    return x
interact(f, x=10);

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-interact',))

In [25]:
def magnify():
    return [dict(selector="th",
                 props=[("font-size", "5pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
           ]

In [33]:
np.random.seed(25)
cmap = cmap=sns.diverging_palette(5, 150, as_cmap=True)
bigdf = pd.DataFrame(np.random.randn(20, 25)).cumsum().head(10)

In [34]:
bigdf.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '1pt'})\
    .set_caption("Hover to magnify")\
    .set_precision(2)\
    .set_table_styles(magnify())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,0.23,1,-0.84,-0.59,-0.96,-0.22,-0.62,1.8,-2.1,0.87,-0.92,-0.23,2.2,-1.3,0.076,-1.2,1.2,-1,1.1,-0.42,2.3,-2.6,2.8,0.68,-1.6
1,-1.7,1.6,-1.1,-1.1,1,0.0037,-2.5,3.4,-1.7,1.3,-0.52,-0.015,1.5,-1.1,-1.9,-1.1,-0.68,-0.81,0.35,-0.055,1.8,-2.8,2.3,0.78,0.44
2,-0.65,3.2,-1.8,0.52,2.2,-0.37,-3,3.7,-1.9,2.5,0.21,-0.24,-0.1,-0.78,-3,-0.82,-0.21,-0.23,0.86,-0.68,1.4,-4.9,3,1.9,0.61
3,-1.6,3.7,-2.3,0.43,4.2,-0.43,-3.9,4.2,-2.1,1.1,0.12,0.6,-0.89,0.27,-3.7,-2.7,-0.31,-1.6,1.4,-1.8,0.91,-5.8,2.8,2.1,0.28
4,-3.3,4.5,-1.9,-1.7,5.2,-1,-3.8,4.7,-0.72,1.1,-0.18,0.83,-0.22,-1.1,-4.3,-2.9,-0.97,-1.8,1.5,-1.8,2.2,-6.3,3.3,2.5,2.1
5,-0.84,4.2,-1.7,-2,5.3,-0.99,-4.1,3.9,-1.1,-0.94,1.2,0.087,-1.8,-0.11,-4.5,-0.85,-2.1,-1.4,0.8,-1.6,1.5,-6.5,2.8,2.1,3.8
6,-0.74,5.4,-2.1,-1.1,4.2,-1.8,-3.2,3.8,-3.2,-1.2,0.34,0.57,-1.8,0.54,-4.4,-1.8,-4,-2.6,-0.2,-4.7,1.9,-8.5,3.3,2.5,5.8
7,-0.44,4.7,-2.3,-0.21,5.9,-2.6,-1.8,5.5,-4.5,-3.2,-1.7,0.18,0.11,0.036,-6,-0.45,-6.2,-3.9,0.71,-3.9,0.67,-7.3,3,3.4,6.7
8,0.92,5.8,-3.3,-0.65,6,-3.2,-1.8,5.6,-3.5,-1.3,-1.6,0.82,-2.4,-0.4,-6.1,-0.52,-6.6,-3.5,-0.043,-4.6,0.51,-5.8,3.2,2.4,5.1
9,0.38,5.5,-4.5,-0.8,7.1,-2.6,-0.44,5.3,-2,-0.33,-0.8,0.26,-3.4,-0.82,-6.1,-2.6,-8.5,-4.5,0.41,-4.7,1.9,-6.9,2.1,3,5.2
